<a href="https://colab.research.google.com/github/afterdeath9/PAFAssignment2/blob/main/buyer_seller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload following files first,
* `Mock data buyer (Wholesaler).xlsx`
* `Mock data seller( Farmer).xlsx`

In [ ]:
import openpyxl
import pandas as pd
from sklearn import preprocessing

In [ ]:
def get_raw(sheet_obj, raw, columns):

    lst = []

    for column in columns:
      cell_obj = sheet_obj.cell(row=raw, column=column)
      lst.append(cell_obj.value)

    return lst

def get_column(sheet_obj, start_raw_num, columns):

    raw_num = start_raw_num
    lst = []

    while(True):

        cell_obj = sheet_obj.cell(row=raw_num, column=columns[0])
        if cell_obj.value == None:
          break
        raw = get_raw(sheet_obj, raw_num, columns)
        lst.append(raw)
        raw_num += 1

    return lst

In [ ]:
buyer_excel_path = "//content//Mock data buyer (Wholesaler).xlsx"
seller_excel_path = "//content//Mock data seller( Farmer).xlsx"

wb_obj = openpyxl.load_workbook(buyer_excel_path)
sheet_name = wb_obj.sheetnames[0]
sheet_obj = wb_obj[sheet_name]
buyers_lst = get_column(sheet_obj, 2, [i for i in range(1, 7)])

wb_obj = openpyxl.load_workbook(seller_excel_path)
sheet_name = wb_obj.sheetnames[0]
sheet_obj = wb_obj[sheet_name]
sellers_lst = get_column(sheet_obj, 2, [i for i in range(1, 7)])

In [ ]:
buyers_sellers_lst = []

for lst in buyers_lst:
  new_lst = ['buyer']
  for i in lst:
    new_lst.append(i)
  buyers_sellers_lst.append(new_lst)

for lst in sellers_lst:
  new_lst = ['seller']
  for i in lst:
    new_lst.append(i)
  buyers_sellers_lst.append(new_lst)

In [ ]:
buyers_sellers_table = pd.DataFrame(data=buyers_sellers_lst,columns=['buyer/seller', 'name', 'district', 'city', 'crop', 'yield','price'])

In [ ]:
buyers_sellers_table

,buyer/seller,name,district,city,crop,yield,price
0,buyer,Jayasundara M D Y L,Colombo,Colombo,Beans,126,160
1,buyer,Kishanth S,Colombo,Dehiwala-Mount Lavinia,Carrot,22,150
2,buyer,Peiris W. U. P.,Colombo,Moratuwa,Cabbage,14,65
3,buyer,Bhagya B.A.I.,Colombo,Sri Jayawardenapura Kotte,Tomato,138,80
4,buyer,Pathirana P.M.C,Gampaha,Negombo,Brinjal,64,200
...,...,...,...,...,...,...,...
215,seller,Mayadunna M. D. T. P.,Ratnapura,Balangoda,Beans,57,125
216,seller,Bhaggya G. V. C.,Hambantota,Hambantota,Carrot,70,115
217,seller,Tharinda S.M.C.,Hambantota,Tangalle,Cabbage,109,28
218,seller,Abraar S.F.M.,Moneragala,Moneragala,Tomato,158,95


In [ ]:
min_num_clusters = len(buyers_sellers_table['crop'].unique())

In [ ]:
buyers_sellers_table['crop'].unique()

array(['Beans', 'Carrot', 'Cabbage', 'Tomato', 'Brinjal', 'Pumpkin',
       'Snake gourd', 'Green Chilli', 'Lime', 'Red Onion (Local)',
       'Red Onion (lmp)', 'Big Onion (Local)', 'Big Onion (Imp)',
       'Potato (Local)', 'Potato (Imp)', 'Dried Chilli (Imp)',
       'Coconut (Avg.)', 'Dhal', 'Banana (Sour)', 'Papaw', 'Pineapple',
       'Apple (Imp)', 'Orange (Imp)', 'Samba', 'Nadu', 'Kekulu (White)',
       'Kekulu (Red)', 'Ponni Samba (Imp)', 'Nadu (Imp)',
       'Kekulu (White) (Imp)'], dtype=object)

In [ ]:
buyers_sellers_table['district'].unique()

array(['Colombo', 'Gampaha', 'Kandy', 'Ampara', 'Vavuniya', 'Galle',
       'Trincomalee', 'Batticaloa', 'Jaffna', 'Matale', 'Anuradhapura',
       'Ratnapura', 'Badulla', 'Matara', 'Puttalam', 'Kalutara', 'Mannar',
       'Kurunegala', 'Nuwara Eliya', 'Kegalle', 'Hambantota',
       'Moneragala'], dtype=object)

In [ ]:
X = buyers_sellers_table.iloc[:,4:].values
le = preprocessing.LabelEncoder()
X[:, 0] = le.fit_transform(X[:, 0])

In [ ]:
X[:, 0] = X[:, 0] + 1
X[:, 0] = X[:, 0] * 10000

In [ ]:
le.inverse_transform([int((X[0, 0]/10000) - 1)])

array(['Beans'], dtype=object)

In [ ]:
n_clusters = int(1.5 * min_num_clusters)

In [ ]:
# from sklearn.cluster import AgglomerativeClustering

# hc = AgglomerativeClustering(n_clusters = n_clusters, affinity = 'euclidean', linkage = 'ward')
# y_pred = hc.fit_predict(X)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = n_clusters, init = "k-means++")
y_pred = kmeans.fit_predict(X)

In [ ]:
matchs = dict()
for i, group in enumerate(y_pred):
  if group not in matchs.keys():
    matchs[group] = [i]
  else:
    matchs[group].append(i)

In [ ]:
buyers_sellers_table.iloc[0,:]

buyer/seller                   buyer
name            Jayasundara M D Y L 
district                     Colombo
city                         Colombo
crop                           Beans
yield                            126
price                            160
Name: 0, dtype: object

In [ ]:
ex_crop = 'Beans'
for key in matchs.keys():
  is_newline = False
  for i in matchs[key]:
    if ex_crop == buyers_sellers_table.iloc[i,4]:
      print(
          buyers_sellers_table.iloc[i,4], 
          buyers_sellers_table.iloc[i,0], 
          buyers_sellers_table.iloc[i,5], 
          buyers_sellers_table.iloc[i,6])
      is_newline = True
  if is_newline: print()

Beans buyer 126 160
Beans seller 122 160
Beans seller 150 145
Beans seller 97 128
Beans seller 98 200

Beans buyer 13 320
Beans buyer 50 295
Beans seller 90 240
Beans seller 146 260

Beans buyer 51 160
Beans buyer 22 135
Beans buyer 54 160
Beans buyer 57 145
Beans seller 57 125

